In [1]:
%load_ext autoreload
%autoreload 2

# Goal: Run facility choice causal model optimization to find input parameters for Vivarium

The optimization results are output in a `.csv` file at the end of the notebook.

## Requirements:

If you want to run this notebook or any of the code in the
`birth_facility.py` or `solution_finding.py` modules, you will need the right environment.
You can create it with these steps (on 64-bit Linux):

- `conda create --name facility_choice --file environment_lock_conda.txt`
- `conda activate facility_choice`
- `pip install -r environment_lock_pip.txt`

If you're not on 64-bit Linux, replacing the first command with `conda create --name facility_choice python=3.10`
should approximately recreate the environment.

If you want to update all the packages in the environment, create it using `conda env create --name facility_choice --file environment.yaml` (no need for any `pip` commands).
Then, run the following commands inside the environment you created to update the lock files:

- `conda list --explicit > environment_lock_conda.txt`
- `pip freeze | grep -v 'file:///' | grep -v '\-e' > environment_lock_pip.txt`

You will also need to be on the IHME cluster to run this notebook, as the causal model
nanosim loads LBWSG data from an Artifact for the MNCNH Portfolio model.

In [2]:
import pandas as pd

import birth_facility as bf
import solution_finding as sf

!date
!whoami
!uname -a
!pwd
!python --version
!conda info --envs | grep '\*'
!conda list | grep -e pandas -e numpy -e scipy -e statsmodels -e matplotlib -e gbd

Wed Oct 29 13:43:58 PDT 2025
zmbc
Linux long-slurm-sarchive-p0064 5.4.0-216-generic #236-Ubuntu SMP Fri Apr 11 19:53:21 UTC 2025 x86_64 x86_64 x86_64 GNU/Linux
/mnt/share/homes/zmbc/src/vivarium_gates_mncnh/src/vivarium_gates_mncnh/data/facility_choice
Python 3.10.19
facility_choice       *  /mnt/share/homes/zmbc/mambaforge/envs/facility_choice
gbd                       4.37.1                   pypi_0    pypi
gbd-mapping               5.0.0                    pypi_0    pypi
matplotlib-inline         0.1.7                    pypi_0    pypi
numpy                     1.26.4                   pypi_0    pypi
pandas                    1.5.3                    pypi_0    pypi
pandas-stubs              2.2.3.250308             pypi_0    pypi
scipy                     1.15.3                   pypi_0    pypi
statsmodels               0.14.5                   pypi_0    pypi


# Load data

The input data comes from two sources:

- An artifact for the `vivarium_gates_mncnh` simulation, which is used for population structure, LBWSG exposure and RRs, ANC and IFD coverage, etc.
- The file `facility_choice_data.xlsx` in this folder (downloaded from
  the [master version on
  Sharepoint](https://uwnetid.sharepoint.com/:x:/r/sites/ihme_simulation_science_team/Shared%20Documents/Research/BMGF_MNCH/MNCNH%20portfolio%20products/01_Planning/facility_choice_data.xlsx?d=wf3b3dd5f641f413ba3537c4ca3364cdf&csf=1&web=1&e=cNf9X7)),
  which contains all the other country-specific data to run the model

In [3]:
data_e = bf.BirthFacilityChoiceData('ethiopia')
data_n = bf.BirthFacilityChoiceData('nigeria')
data_p = bf.BirthFacilityChoiceData('pakistan')

# Create models for all 3 countries

Each of the three model objects defines the causal model nanosim that
will be run repeatedly by the `OptimalSolutionFinder.find_solutions`
optimization routine below.

In [4]:
# Choose a population size that's large enough to get stochastically
# stable results, but small enough that the optimization isn't too slow
pop_size = 100_000
# In Vivarium, we will be using the 2-facility-type model (at-home vs.
# in-facility)
num_facility_types = 2

# The random seeds were generated by calling numpy.random.SeedSequence()
# and then copying the generated entropy
model_e = bf.BirthFacilityModelWithUltrasoundAndSimpleGAError(
    data_e, pop_size, num_facility_types,
    seed=97269740763240770980627172105157383126
)
model_n = bf.BirthFacilityModelWithUltrasoundAndSimpleGAError(
    data_n, pop_size, num_facility_types,
    seed=230426788122306909803111949239871602900
)
model_p = bf.BirthFacilityModelWithUltrasoundAndSimpleGAError(
    data_p, pop_size, num_facility_types,
    seed=150031875166580400207947577126615739298
)

# Create solution finder objects and specify inputs for optimization

In [5]:
# These are the 5 parameters we need to solve for in the optimization
model_e.parameter_data

attribute,correlated_pair,bounds,position
parameter_name,,,
"corr(anc, lbwsg_category)","(anc, lbwsg_category)","(-1, 1)",0
"corr(anc, facility)","(anc, facility)","(-1, 1)",1
"corr(lbwsg_category, facility)","(lbwsg_category, facility)","(-1, 1)",2
prob_home_given_believed_preterm,<NA>,"(0, 1)",3
prob_home_given_believed_term,<NA>,"(0, 1)",4


In [6]:
finder_e = sf.OptimalSolutionFinder(model_e)
finder_n = sf.OptimalSolutionFinder(model_n)
finder_p = sf.OptimalSolutionFinder(model_p)

# 25 initial conditions seems to be sufficient to get close to the
# global minimum
n_points = 25
# Fix the two LBWSG correlations at 0.2 -- According to the above
# parameter_data dataframe, these parameters are in positions 0 and 2 in
# the parameter vector
fixed_x_components = {0: 0.2, 2: 0.2}

# Find solutions

# Ethiopia

In [7]:
%%time
solutions_e = finder_e.find_solutions(n_points, fixed_x_components)
print(solutions_e.sorted_losses)
solutions_e.sorted_x_values

[4.644412909815543e-08, 5.969138572314847e-08, 8.315337773900922e-08, 8.488957192742674e-08, 8.99221054284638e-08, 9.005534817863037e-08, 9.366910314057009e-08, 1.1319770487627778e-07, 1.1328034210666971e-07, 1.271496414823048e-07, 1.30951377186328e-07, 2.9096035392139896e-07, 3.02595883017176e-07, 4.347007356297894e-07, 4.863890897821577e-07, 5.0216004277015e-07, 6.548712579990124e-07, 0.0031613854303733646, 0.01025665276999066, 0.022610107362757703, 0.045750756740703946, 0.06787068631294424, 0.22693948387707863, 0.23892873220764277, inf]
CPU times: user 9min 18s, sys: 6min 36s, total: 15min 55s
Wall time: 8min 49s


parameter_name,"corr(anc, lbwsg_category)","corr(anc, facility)","corr(lbwsg_category, facility)",prob_home_given_believed_preterm,prob_home_given_believed_term
0,0.2,0.684812,0.2,0.308393,0.491023
1,0.2,0.685171,0.2,0.310360,0.490098
2,0.2,0.684949,0.2,0.308732,0.490648
3,0.2,0.684904,0.2,0.310695,0.489901
4,0.2,0.684956,0.2,0.308309,0.490717
5,0.2,0.684438,0.2,0.308898,0.490792
6,0.2,0.684683,0.2,0.308630,0.490714
7,0.2,0.684842,0.2,0.309053,0.490474
8,0.2,0.685193,0.2,0.309994,0.490189
9,0.2,0.683653,0.2,0.309179,0.490739


In [8]:
data_e.targets_2_facility_types.to_frame().join(solutions_e.sorted_targets)

target_probabilities         0         1  \
probability_of given                                                   
in_facility    anc0                     0.183369  0.183938  0.182955   
               anc1                     0.664786  0.664622  0.665032   
preterm        at_home                  0.163579  0.163188  0.164310   
               in_facility              0.163486  0.164068  0.163237   

                                   2         3         4         5         6  \
probability_of given                                                           
in_facility    anc0         0.183328  0.181154  0.184257  0.182527  0.183299   
               anc1         0.664908  0.665057  0.664750  0.664908  0.664532   
preterm        at_home      0.163853  0.164403  0.163228  0.163737  0.163722   
               in_facility  0.163562  0.162144  0.164036  0.162634  0.162681   

                                   7         8  ...        15        16  \
probability_of given                            ...                       
in_facility    anc0         0.183017  0.181495  ...  0.182498  0.183266   
               anc1         0.664720  0.665044  ...  0.664788  0.665450   
preterm        at_home      0.163797  0.164059  ...  0.163431  0.164251   
               in_facility  0.162640  0.162415  ...  0.163966  0.163302   

                                  17        18        19        20        21  \
probability_of given                                                           
in_facility    anc0         0.189814  0.141680  0.310513  0.215063  0.184913   
               anc1         0.686515  0.701967  0.672796  0.813957  0.588752   
preterm        at_home      0.120177  0.094776  0.079240  0.342421  0.305472   
               in_facility  0.195361  0.213716  0.221471  0.080457  0.020714   

                                  22        23        24  
probability_of given                                      
in_facility    anc0         0.648606  0.350205  0.183266  
               anc1         0.734325  0.240003  0.665450  
preterm        at_home      0.509663  0.025509  0.164251  
               in_facility  0.025118  0.542454  0.163302  

[4 rows x 26 columns]

In [9]:
data_e.input_probabilities.to_frame().join(solutions_e.sorted_pop_proportions)

,input_probability,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24
subpopulation,,,,,,,,,,,,,,,,,,,,,
Male,0.515312,0.512060,0.515200,0.512060,0.516510,0.512060,0.516510,0.516510,0.516510,0.516510,...,0.512060,0.512060,0.515200,0.51520,0.515200,0.51206,0.516510,0.515200,0.51520,0.512060
Female,0.484688,0.487940,0.484800,0.487940,0.483490,0.487940,0.483490,0.483490,0.483490,0.483490,...,0.487940,0.487940,0.484800,0.48480,0.484800,0.48794,0.483490,0.484800,0.48480,0.487940
anc0,0.219890,0.219150,0.219180,0.219170,0.219150,0.219150,0.219200,0.219150,0.219160,0.219180,...,0.219180,0.219190,0.219900,0.21965,0.219540,0.21975,0.219400,0.220550,0.21930,0.219190
anc1,0.780110,0.780850,0.780820,0.780830,0.780850,0.780850,0.780800,0.780850,0.780840,0.780820,...,0.780820,0.780810,0.780100,0.78035,0.780460,0.78025,0.780600,0.779450,0.78070,0.780810
preterm,0.163527,0.163680,0.163710,0.163690,0.163140,0.163680,0.163120,0.163140,0.163150,0.163140,...,0.163730,0.163720,0.163580,0.16363,0.163620,0.16367,0.163050,0.163010,0.16207,0.163720
term,0.836473,0.836320,0.836290,0.836310,0.836860,0.836320,0.836880,0.836860,0.836850,0.836860,...,0.836270,0.836280,0.836420,0.83637,0.836380,0.83633,0.836950,0.836990,0.83793,0.836280
at_home,0.441073,0.440720,0.440630,0.440640,0.440990,0.440550,0.440830,0.440930,0.440850,0.440940,...,0.440920,0.440240,0.422710,0.42110,0.406740,0.31765,0.499850,0.284580,0.73583,0.440240
in_facility,0.558927,0.559280,0.559370,0.559360,0.559010,0.559450,0.559170,0.559070,0.559150,0.559060,...,0.559080,0.559760,0.577290,0.57890,0.593260,0.68235,0.500150,0.715420,0.26417,0.559760
BEmONC,0.089922,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Nigeria

In [10]:
%%time
solutions_n = finder_n.find_solutions(n_points, fixed_x_components)
print(solutions_n.sorted_losses)
solutions_n.sorted_x_values

[1.70550145783821e-07, 1.771658381111152e-07, 1.9269934625754104e-07, 2.0048268378758394e-07, 2.1465928357056185e-07, 2.288255512850057e-07, 2.362559864410585e-07, 2.712235158686127e-07, 3.0154254049286067e-07, 3.5732864511750506e-07, 3.771023439069765e-07, 4.1662472760606306e-07, 4.2652576226487326e-07, 4.4492413941998876e-07, 4.534828264013413e-07, 4.857811316538729e-07, 4.878999452229138e-07, 5.814478952759217e-07, 6.092067910135057e-07, 0.017369803610519652, 0.02177290634222928, 0.02232548748315033, 0.05556009666635786, 0.07257413041233307, 0.17159687310536587]
CPU times: user 9min 39s, sys: 6min 35s, total: 16min 15s
Wall time: 8min 51s


parameter_name,"corr(anc, lbwsg_category)","corr(anc, facility)","corr(lbwsg_category, facility)",prob_home_given_believed_preterm,prob_home_given_believed_term
0,0.2,0.416376,0.2,0.383322,0.523173
1,0.2,0.416059,0.2,0.384392,0.522588
2,0.2,0.416513,0.2,0.383719,0.523154
3,0.2,0.415856,0.2,0.384100,0.523246
4,0.2,0.415313,0.2,0.384721,0.522662
5,0.2,0.416212,0.2,0.383219,0.523048
6,0.2,0.415647,0.2,0.384408,0.522459
7,0.2,0.416437,0.2,0.383991,0.522611
8,0.2,0.414308,0.2,0.384382,0.522535
9,0.2,0.415387,0.2,0.382831,0.523144


In [11]:
data_n.targets_2_facility_types.to_frame().join(solutions_n.sorted_targets)

target_probabilities         0         1  \
probability_of given                                                   
in_facility    anc0                     0.309069  0.308668  0.308494   
               anc1                     0.584168  0.583827  0.584501   
preterm        at_home                  0.172663  0.173383  0.172831   
               in_facility              0.153000  0.153206  0.153377   

                                   2         3         4         5         6  \
probability_of given                                                           
in_facility    anc0         0.308135  0.308246  0.308231  0.308872  0.308270   
               anc1         0.584357  0.584081  0.583191  0.583917  0.583487   
preterm        at_home      0.172530  0.172638  0.174045  0.173470  0.173963   
               in_facility  0.153615  0.153527  0.152710  0.153206  0.152759   

                                   7         8  ...        15        16  \
probability_of given                            ...                       
in_facility    anc0         0.309057  0.309788  ...  0.309204  0.309557   
               anc1         0.584715  0.584227  ...  0.582976  0.584708   
preterm        at_home      0.173181  0.173292  ...  0.173626  0.172654   
               in_facility  0.153018  0.153063  ...  0.153112  0.153580   

                                  17        18        19        20        21  \
probability_of given                                                           
in_facility    anc0         0.309470  0.307534  0.320036  0.470207  0.446172   
               anc1         0.584740  0.585018  0.683636  0.455138  0.433218   
preterm        at_home      0.173001  0.172161  0.088597  0.169963  0.191889   
               in_facility  0.153583  0.153939  0.214700  0.154684  0.126629   

                                  22        23        24  
probability_of given                                      
in_facility    anc0         0.309011  0.303311  0.659852  
               anc1         0.541066  0.693351  0.723249  
preterm        at_home      0.294812  0.367246  0.498225  
               in_facility  0.021391  0.022463  0.025403  

[4 rows x 26 columns]

In [12]:
data_n.input_probabilities.to_frame().join(solutions_n.sorted_pop_proportions)

,input_probability,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24
subpopulation,,,,,,,,,,,,,,,,,,,,,
Male,0.512450,0.509000,0.510960,0.510960,0.510960,0.509000,0.509000,0.509000,0.510960,0.510960,...,0.509000,0.510960,0.509000,0.510960,0.51096,0.512480,0.512480,0.509000,0.512480,0.511540
Female,0.487550,0.491000,0.489040,0.489040,0.489040,0.491000,0.491000,0.491000,0.489040,0.489040,...,0.491000,0.489040,0.491000,0.489040,0.48904,0.487520,0.487520,0.491000,0.487520,0.488460
anc0,0.257001,0.257040,0.256990,0.257030,0.256970,0.256950,0.257000,0.256950,0.257040,0.256950,...,0.256950,0.256980,0.257020,0.256980,0.25769,0.256270,0.257210,0.256690,0.257360,0.256300
anc1,0.742999,0.742960,0.743010,0.742970,0.743030,0.743050,0.743000,0.743050,0.742960,0.743050,...,0.743050,0.743020,0.742980,0.743020,0.74231,0.743730,0.742790,0.743310,0.742640,0.743700
preterm,0.162567,0.163030,0.162840,0.162820,0.162830,0.163110,0.163070,0.163090,0.162820,0.162900,...,0.163110,0.162850,0.163020,0.162800,0.16299,0.162950,0.163400,0.163160,0.162800,0.163940
term,0.837433,0.836970,0.837160,0.837180,0.837170,0.836890,0.836930,0.836910,0.837180,0.837100,...,0.836890,0.837150,0.836980,0.837200,0.83701,0.837050,0.836600,0.836840,0.837200,0.836060
at_home,0.486533,0.486900,0.486430,0.486640,0.486800,0.487460,0.486770,0.487230,0.486140,0.486290,...,0.487370,0.486000,0.486010,0.486290,0.41006,0.541000,0.563450,0.518500,0.407030,0.293000
in_facility,0.513467,0.513100,0.513570,0.513360,0.513200,0.512540,0.513230,0.512770,0.513860,0.513710,...,0.512630,0.514000,0.513990,0.513710,0.58994,0.459000,0.436550,0.481500,0.592970,0.707000
BEmONC,0.002271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Pakistan

In [13]:
%%time
solutions_p = finder_p.find_solutions(n_points, fixed_x_components)
print(solutions_p.sorted_losses)
solutions_p.sorted_x_values

[1.0786234430071318e-06, 1.1279055669444205e-06, 1.1439547464853206e-06, 1.331894440181891e-06, 1.4211062844271538e-06, 1.442338447454894e-06, 1.548766642045507e-06, 1.5746702507479782e-06, 1.6251014612889136e-06, 1.6721257083407437e-06, 1.6931060726754765e-06, 1.6954645770672627e-06, 1.7171150270556268e-06, 1.834133195544041e-06, 2.179008372182878e-06, 2.6967907386099554e-06, 2.7634317768532313e-06, 0.0003969708871405242, 0.0018321657501252364, 0.00539541899788043, 0.0061057370829255, 0.009207238470684165, 0.08120101046540384, 0.09751344906694914, inf]
CPU times: user 8min 16s, sys: 5min 50s, total: 14min 7s
Wall time: 7min 45s


parameter_name,"corr(anc, lbwsg_category)","corr(anc, facility)","corr(lbwsg_category, facility)",prob_home_given_believed_preterm,prob_home_given_believed_term
0,0.2,0.350349,0.2,0.170056,0.244330
1,0.2,0.350499,0.2,0.169809,0.244493
2,0.2,0.350793,0.2,0.169741,0.244344
3,0.2,0.351661,0.2,0.169263,0.244200
4,0.2,0.350623,0.2,0.169819,0.245177
5,0.2,0.349008,0.2,0.170494,0.245164
6,0.2,0.352608,0.2,0.168673,0.244002
7,0.2,0.351316,0.2,0.167658,0.245581
8,0.2,0.352414,0.2,0.168731,0.244073
9,0.2,0.351443,0.2,0.168965,0.244943


In [14]:
data_p.targets_2_facility_types.to_frame().join(solutions_p.sorted_targets)

target_probabilities         0         1  \
probability_of given                                                   
in_facility    anc0                     0.556322  0.556515  0.556116   
               anc1                     0.795407  0.795395  0.795601   
preterm        at_home                  0.194529  0.194303  0.195567   
               in_facility              0.155164  0.154127  0.153504   

                                   2         3         4         5         6  \
probability_of given                                                           
in_facility    anc0         0.556722  0.556765  0.556663  0.558035  0.556676   
               anc1         0.795657  0.796007  0.794600  0.794193  0.795806   
preterm        at_home      0.194351  0.194045  0.194528  0.194418  0.194141   
               in_facility  0.154088  0.154154  0.153549  0.153592  0.153756   

                                   7         8  ...        15        16  \
probability_of given                            ...                       
in_facility    anc0         0.557769  0.557207  ...  0.556676  0.556397   
               anc1         0.794874  0.795828  ...  0.796304  0.793713   
preterm        at_home      0.193156  0.194598  ...  0.192830  0.195003   
               in_facility  0.154011  0.153629  ...  0.154141  0.153367   

                                  17        18        19        20        21  \
probability_of given                                                           
in_facility    anc0         0.537189  0.560381  0.562682  0.650033  0.610533   
               anc1         0.811648  0.839834  0.868907  0.770531  0.701492   
preterm        at_home      0.198766  0.209301  0.179384  0.162348  0.239806   
               in_facility  0.153254  0.152419  0.159834  0.165647  0.130724   

                                  22        23        24  
probability_of given                                      
in_facility    anc0         0.604307  0.662180  0.556663  
               anc1         0.871353  0.764731  0.794600  
preterm        at_home      0.558041  0.548881  0.194528  
               in_facility  0.093310  0.041416  0.153549  

[4 rows x 26 columns]

In [15]:
data_p.input_probabilities.to_frame().join(solutions_p.sorted_pop_proportions)

,input_probability,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24
subpopulation,,,,,,,,,,,,,,,,,,,,,
Male,0.519811,0.51732,0.515460,0.517320,0.517320,0.519420,0.519420,0.519420,0.519420,0.51942,...,0.519420,0.519420,0.517320,0.519420,0.515460,0.517320,0.51732,0.519420,0.515460,0.519420
Female,0.480189,0.48268,0.484540,0.482680,0.482680,0.480580,0.480580,0.480580,0.480580,0.48058,...,0.480580,0.480580,0.482680,0.480580,0.484540,0.482680,0.48268,0.480580,0.484540,0.480580
anc0,0.075919,0.07529,0.075290,0.075280,0.075310,0.075270,0.075300,0.075340,0.075300,0.07534,...,0.075340,0.075270,0.075560,0.075520,0.075620,0.074950,0.07500,0.075690,0.075780,0.075270
anc1,0.924081,0.92471,0.924710,0.924720,0.924690,0.924730,0.924700,0.924660,0.924700,0.92466,...,0.924660,0.924730,0.924440,0.924480,0.924380,0.925050,0.92500,0.924310,0.924220,0.924730
preterm,0.163933,0.16307,0.162860,0.163040,0.163010,0.162700,0.162720,0.162730,0.162740,0.16273,...,0.162720,0.162700,0.162770,0.162730,0.162850,0.164860,0.16403,0.162490,0.164750,0.162700
term,0.836067,0.83693,0.837140,0.836960,0.836990,0.837300,0.837280,0.837270,0.837260,0.83727,...,0.837280,0.837300,0.837230,0.837270,0.837150,0.835140,0.83597,0.837510,0.835250,0.837300
at_home,0.222744,0.22259,0.222430,0.222330,0.222010,0.223310,0.223590,0.222210,0.222980,0.22215,...,0.221750,0.224150,0.209090,0.181270,0.154250,0.238500,0.30533,0.148860,0.243040,0.223310
in_facility,0.777256,0.77741,0.777570,0.777670,0.777990,0.776690,0.776410,0.777790,0.777020,0.77785,...,0.778250,0.775850,0.790910,0.818730,0.845750,0.761500,0.69467,0.851140,0.756960,0.776690
BEmONC,0.264677,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Collect optimization results from the 3 countries

Select the solution with the lowest log loss for each country. The above
solutions dataframes are already sorted by log loss, so we just select
the first row of each.

In [16]:
solutions_by_location = {
    'Ethiopia': solutions_e,
    'Nigeria': solutions_n,
    'Pakistan': solutions_p,
}
best_solutions = pd.concat({
    location: solutions.sorted_x_values.loc[0]
    for location, solutions in solutions_by_location.items()
}, axis=1)
best_solutions

,Ethiopia,Nigeria,Pakistan
parameter_name,,,
"corr(anc, lbwsg_category)",0.200000,0.200000,0.200000
"corr(anc, facility)",0.684812,0.416376,0.350349
"corr(lbwsg_category, facility)",0.200000,0.200000,0.200000
prob_home_given_believed_preterm,0.308393,0.383322,0.170056
prob_home_given_believed_term,0.491023,0.523173,0.244330


# Save results to file

In [17]:
output_filename = 'facility_choice_optimization_results.csv'
best_solutions.to_csv(output_filename)
pd.read_csv(output_filename)

,parameter_name,Ethiopia,Nigeria,Pakistan
0,"corr(anc, lbwsg_category)",0.200000,0.200000,0.200000
1,"corr(anc, facility)",0.684812,0.416376,0.350349
2,"corr(lbwsg_category, facility)",0.200000,0.200000,0.200000
3,prob_home_given_believed_preterm,0.308393,0.383322,0.170056
4,prob_home_given_believed_term,0.491023,0.523173,0.244330
